In [ ]:
# !pip3 install Pillow --break-system-packages
# !pip3 install rembg --break-system-packages
# !pip3 install piexif --break-system-packages
# !pip3 install onnxruntime-gpu --break-system-packages
# !pip3 install onnxruntime --break-system-packages

# Libraries

In [1]:
import os
import piexif
import cv2
import itertools

import numpy as np
import pandas as pd

from collections import Counter
from tqdm import tqdm
from pprint import pprint
from pathlib import Path

from rembg import remove 
from PIL import Image 

# Functions

In [2]:
# Get a list of all files and folders in the specified directory
def get_folders_in_directory(directory):
    items = os.listdir(directory)
    
    # Filter out the items that are folders
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]
    
    return folders

def list_directory_contents(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    
    return items

In [3]:
# Specify the directory path
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_50_3'

# Get the list of folders in the directory
folders = get_folders_in_directory(directory_path)
# folders = [x for x in folders if 'Subject' in x]
folders.sort()

# Print the list of folders
pprint(folders)

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '8',
 '9']


In [4]:
subject_dic = {}

for folder in folders:
    folder_path = os.path.join(directory_path, folder)
    
    # Get the list of files and folders in the directory
    contents = list_directory_contents(folder_path)
    contents = [x for x in contents if ':Zone.Identifier' not in x]
    
    subject_dic[folder] = contents
    
    for content in contents:
        folder_path_content = os.path.join(directory_path, folder, content)
        
        images = list_directory_contents(folder_path_content)
        images = [x for x in images if ':Zone.Identifier' not in x]
        
        print()
        print('#'*60)
        print(folder_path_content)
        print(images)
    


############################################################
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_50_3/1/1D0L
['frame_0003.jpeg', 'frame_0028.jpeg', 'frame_0005.jpeg', 'frame_0023.jpeg', 'frame_0013.jpeg', 'frame_0000.jpeg', 'frame_0012.jpeg', 'frame_0004.jpeg', 'frame_0021.jpeg', 'frame_0025.jpeg', 'frame_0019.jpeg', 'frame_0008.jpeg', 'frame_0029.jpeg', 'frame_0020.jpeg', 'frame_0002.jpeg', 'frame_0024.jpeg', 'frame_0022.jpeg', 'frame_0011.jpeg', 'frame_0027.jpeg', 'frame_0017.jpeg', 'frame_0009.jpeg', 'frame_0016.jpeg', 'frame_0006.jpeg', 'frame_0007.jpeg', 'frame_0018.jpeg', 'frame_0001.jpeg', 'frame_0010.jpeg', 'frame_0026.jpeg', 'frame_0015.jpeg', 'frame_0014.jpeg']

############################################################
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_50_3/1/1D0R
['frame_0003.jpeg', 'frame_0005.jpeg', 'frame_0013.jpeg', 'frame_0000.jpeg', 'frame_0012.jpeg', 'frame_0004.jpeg', 'frame_0008.jpeg', 'fram

In [5]:
subject_dic

{'1': ['1D0L', '1D0R'],
 '10': ['10D2R', '10D2L', '10D0R', '10D1R', '10D0L', '10D1L'],
 '11': ['11D2R', '11D1R', '11D1L', '11D0L', '11D0R', '11D2L'],
 '12': ['12D0L', '12D0R'],
 '13': ['13D5L', '13D1L', '13D2L', '13D0L'],
 '14': ['14D1L', '14D0L', '14D1R', '14D0R'],
 '15': ['15D2L', '15D0L', '15D1L', '15D2R', '15D1R', '15D0R'],
 '16': ['16D0L', '16D1R', '16D1L', '16D0R'],
 '17': ['17D1L',
  '17D1R',
  '17D0L',
  '17D2R',
  '17D4R',
  '17D3L',
  '17D3R',
  '17D2L',
  '17D0R',
  '17D4L'],
 '18': ['18D0R', '18D1R', '18D0L', '18D1L'],
 '19': ['19D0R',
  '19D1R',
  '19D0L',
  '19D1L',
  '19D3L',
  '19D3R',
  '19D2L',
  '19D2R'],
 '2': ['2D4L', '2D5R', '2D5L', '2D0R', '2D4R', '2D0L'],
 '20': ['20D0L', '20D2L', '20D1L', '20D3L'],
 '21': ['21D0R', '21D0L'],
 '22': ['22D2R', '22D0L_2', '22D2L', '22D1L', '22D0L', '22D0R', '22D1R'],
 '23': ['23D0R', '23D1R', '23D1L', '23D0L'],
 '24': ['24D3L', '24D2L', '24D2R', '24D3R', '24D0L', '24D0R'],
 '25': ['25D0R',
  '25D3L',
  '25D1R',
  '25D2L',
  '25D1L

In [8]:
subject_dic = {
    '71': ['71D0R', '71D0L', '71D1R', '71D1L']
}

# Using traditional models

In [10]:
list_algorithm = [
    '',
    # '_colmap',
    # 'mve',
    # 'vggsfm'
    ]

# list_scale_percent = [50]
# list_max_frames = [3]

list_scale_percent = [100]
list_max_frames = [5]

for scale_percent, max_frames in itertools.product(list_scale_percent, list_max_frames):
    print(f'Scale: {scale_percent}, Max Frames: {max_frames}')
    
    for subject in subject_dic.keys():    
        print('Subject', subject)
        folders = subject_dic[subject]
        
        for folder in folders:
            # When the input is raw
            # Convert to no bg (with/without resize)        
            path_input = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input'
            path_output = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input'

            path_input = os.path.join(f'{path_input}_{scale_percent}_{max_frames}', subject, folder)
            folder_path = Path(path_input)
                            
            ##################################################################
            # Loop through all files in the folder
            ##################################################################
            for file_path in tqdm(list(folder_path.iterdir()), total=len(list(folder_path.iterdir()))):
                if (
                    file_path.is_file() and 
                    ('.jpeg' in file_path.name) and 
                    (':Zone.Identifier' not in file_path.name)
                    ):

                    input_image_name = file_path.name
                    
                    file_path_input = os.path.join(path_input, input_image_name)
                    
                    # Processing the image 
                    # Removing the background from the given Image
                    input_img = Image.open(file_path_input)
                    input_img = remove(input_img) 
                    input_img = input_img.convert("RGB")

                    for algorithm in list_algorithm:
                        if algorithm == '':
                            file_path_output = os.path.join(f'{path_output}{algorithm}_{scale_percent}_{max_frames}_no_bg', subject, folder)  
                        else:
                            file_path_output = os.path.join(f'{path_output}{algorithm}_{scale_percent}_{max_frames}_no_bg', subject, folder, 'images')  
                    
                        if not os.path.exists(file_path_output):
                            os.makedirs(file_path_output)
                    
                        # print('')
                        # print(file_path_input)
                        # print(file_path_output)
                    
                        # Saving data
                        input_img.save(os.path.join(file_path_output, input_image_name))
                    
            

Scale: 100, Max Frames: 5
Subject 71


100%|██████████| 55/55 [00:44<00:00,  1.25it/s]
